<a href="https://colab.research.google.com/github/SadoRuin/maskDetective/blob/yolo/Mask_Detective_YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Dataset and YOLOv5

Upload from local (kaggle.json)

In [ ]:
# !pip install kaggle
# from google.colab import files
# files.upload()

Copy from GoogleDrive (kaggle.json)

In [ ]:
!cp /content/drive/MyDrive/kaggle/kaggle.json /content/

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions list

In [ ]:
%ls

Download Dataset from kaggle

In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection
%ls

In [ ]:
!unzip face-mask-detection.zip

Setup YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5m.pt
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l.pt
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5x.pt

In [ ]:
%ls

In [ ]:
!mkdir -p dataset/mask/images
!mkdir -p dataset/mask/labels

In [ ]:
!cp -rf ../images/* dataset/mask/images

In [ ]:
!mkdir -p dataset/images dataset/labels

# Convert Labels

In [ ]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile

In [ ]:
FILE_ROOT = "../"
IMAGE_PATH = FILE_ROOT + "images"  
ANNOTATIONS_PATH = FILE_ROOT + "annotations"

DATA_ROOT = "dataset/"
LABELS_ROOT = DATA_ROOT + "mask/labels"
IMAGES_ROOT = DATA_ROOT + "mask/images"  

DEST_IMAGES_PATH = "images"
DEST_LABELS_PATH = "labels" 

In [ ]:
classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']

In [ ]:
def cord_converter(size, box):
    """
    convert xml annotation to darknet format coordinates
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def save_file(img_jpg_file_name, size, img_box):
    save_file_name = LABELS_ROOT + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:

        cls_num = classes.index(box[0])

        new_box = cord_converter(size, box[1:])

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()
    
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        # print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
    # print(img_box)

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
files = os.listdir(ANNOTATIONS_PATH)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])

# Split Images Dataset

Make Test Dataset

In [ ]:
# from sklearn.model_selection import train_test_split
# image_list = os.listdir('dataset/mask/images')
# train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=7)
# val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=8)

# print('total =',len(image_list))
# print('train :',len(train_list))
# print('val   :',len(val_list))
# print('test  :',len(test_list))

None Test Dataset

In [ ]:
from sklearn.model_selection import train_test_split
image_list = os.listdir('dataset/mask/images')
train_list, val_list = train_test_split(image_list, test_size=0.1, random_state=7)
test_list = []

print('total =',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))

In [ ]:
def copy_data(file_list, img_labels_root, imgs_source, type):

    root_file = Path(DATA_ROOT + DEST_IMAGES_PATH + '/' + type)
    if not root_file.exists():
        print(f"Path {root_file} is not exit")
        os.makedirs(root_file)

    root_file = Path(DATA_ROOT + DEST_LABELS_PATH + '/' + type)
    if not root_file.exists():
        print(f"Path {root_file} is not exit")
        os.makedirs(root_file)

    for file in file_list:
        img_name = file.replace('.png', '')
        img_src_file = imgs_source + '/' + img_name + '.png'
        label_src_file = img_labels_root + '/' + img_name + '.txt'

        # print(img_sor_file)
        # print(label_sor_file)
        # im = Image.open(rf"{img_sor_file}")
        # im.show()

        # Copy image
        DICT_DIR = DATA_ROOT + DEST_IMAGES_PATH + '/' + type
        img_dict_file = DICT_DIR + '/' + img_name + '.png'

        copyfile(img_src_file, img_dict_file)

        # Copy label
        DICT_DIR = DATA_ROOT + DEST_LABELS_PATH + '/' + type
        img_dict_file = DICT_DIR + '/' + img_name + '.txt'
        copyfile(label_src_file, img_dict_file)

In [ ]:
copy_data(train_list, LABELS_ROOT, IMAGES_ROOT, "train")
copy_data(val_list,   LABELS_ROOT, IMAGES_ROOT, "val")
copy_data(test_list,  LABELS_ROOT, IMAGES_ROOT, "test") if len(test_list) else None

# Create maskDetective.yaml

In [ ]:
!echo "train: dataset/images/train" > data/maskDetective.yaml
!echo "" >> data/maskDetective.yaml
!echo "val:   dataset/images/val" >> data/maskDetective.yaml
!echo "" >> data/maskDetective.yaml
!echo "nc : 3" >> data/maskDetective.yaml
!echo "" >> data/maskDetective.yaml
!echo "names: ['With_Mask', 'Without_Mask', 'Worn_Incorrectly']" >> data/maskDetective.yaml
!echo "" >> data/maskDetective.yaml

!cat data/maskDetective.yaml

# Train YOLOv5

In [ ]:
!python train.py --img 320 --batch 64 --epochs 500 --data data/maskDetective.yaml --cfg models/yolov5l.yaml --weights yolov5l.pt

# Save Result

In [ ]:
!zip -r /content/exp.zip runs/train/exp/*

In [ ]:
!cp /content/exp.zip /content/drive/MyDrive/